PWC-Net-large model training (with multisteps learning rate schedule)
=========================================================

In this notebook, we:
- Use a PWC-Net-large model (with dense and residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a S<sub>long</sub> schedule described in [[2018a]](#2018a)
- The S<sub>long</sub> schedule is borrowed from [[2016a]](#2016a) and looks as follows:

![](img/lr_multisteps.png)

Below, look for `TODO` references and customize this notebook based on your own machine setup.

## Reference

- [2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)
- [2016a]<a name="2016a"></a> Ilg et al. 2016. FlowNet 2.0: Evolution of Optical Flow Estimation with Deep Networks. [[arXiv]](https://arxiv.org/abs/1612.01925) [[PyTorch (Official)]](https://github.com/NVIDIA/flownet2-pytorch) [[TensorFlow]](https://github.com/sampepose/flownet2-tf)

In [1]:
"""
pwcnet_train.ipynb

PWC-Net model training.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-lg-6-2-multisteps-chairsthingsmix
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-lg-6-2-multisteps-chairsthingsmix
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy

from dataset_base import _DEFAULT_DS_TRAIN_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TRAIN_OPTIONS

## TODO: Set this first!

In [2]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/media/EDrive/datasets/'
_FLYINGCHAIRS_ROOT = _DATASET_ROOT + 'FlyingChairs_release'
_FLYINGTHINGS3DHALFRES_ROOT = _DATASET_ROOT + 'FlyingThings3D_HalfRes'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
gpu_devices = ['/device:GPU:0', '/device:GPU:1']
controller = '/device:CPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 8

# Pre-train on `FlyingChairs+FlyingThings3DHalfRes` mix

## Load the dataset

In [3]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TRAIN_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'into_future'
ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [4]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (256, 448)
  scale_preproc        None
  input_channels       3
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           16
  type                 into_future
  mode                 train_with_val
  train size           41282
  val size             1230


## Configure the training

In [5]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TRAIN_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-multisteps-chairsthingsmix/'
nn_opts['batch_size'] = ds_opts['batch_size']
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-large model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

In [6]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['lr_boundaries'] = [400000, 600000, 800000, 1000000, 1200000]
nn_opts['lr_values'] = [0.0001, 5e-05, 2.5e-05, 1.25e-05, 6.25e-06, 3.125e-06]
nn_opts['max_steps'] = 1200000

# Below, we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / ds_opts['batch_size'])
nn_opts['lr_boundaries'] = [int(boundary * 8 / ds_opts['batch_size']) for boundary in nn_opts['lr_boundaries']]

# Debugging changes
nn_opts['max_to_keep'] = 50

In [7]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model towers...
  Building tower_0...
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
  ...tower_0 built.
  Building tower_1...
  ...tower_1 built.
... model towers built.
Initializing model from previous checkpoint ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-310000 to resume training...

INFO:tensorflow:Restoring parameters from ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-310000
... model initialized

Model Configuration:
  verbose                True
  ckpt_dir               ./pwcnet-lg-6-2-multisteps-chairsthingsmix/
  max_to_keep            50
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 256, 448, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [256, 448, 2]
  train_mode             train
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            pyramid
  tb_test_imgs           None
  

## Train the model

In [ ]:
# Train the model
nn.train()

Resume training from step 310001...
2018-09-16 12:14:00 Iter 310100 [Train]: loss=38.81, epe=2.47, lr=0.000025, samples/sec=21.0, sec/step=1.525, eta=5 days, 2:50:07
2018-09-16 12:15:47 Iter 310200 [Train]: loss=36.46, epe=2.28, lr=0.000025, samples/sec=32.1, sec/step=0.997, eta=3 days, 8:16:14
2018-09-16 12:17:38 Iter 310300 [Train]: loss=38.82, epe=2.45, lr=0.000025, samples/sec=31.2, sec/step=1.025, eta=3 days, 10:26:54
2018-09-16 12:19:28 Iter 310400 [Train]: loss=38.15, epe=2.41, lr=0.000025, samples/sec=31.4, sec/step=1.018, eta=3 days, 9:52:43
2018-09-16 12:21:17 Iter 310500 [Train]: loss=36.81, epe=2.32, lr=0.000025, samples/sec=31.5, sec/step=1.015, eta=3 days, 9:39:05
2018-09-16 12:23:06 Iter 310600 [Train]: loss=38.82, epe=2.47, lr=0.000025, samples/sec=31.5, sec/step=1.017, eta=3 days, 9:47:15
2018-09-16 12:24:56 Iter 310700 [Train]: loss=39.47, epe=2.51, lr=0.000025, samples/sec=31.5, sec/step=1.015, eta=3 days, 9:34:53
2018-09-16 12:26:46 Iter 310800 [Train]: loss=38.71, 

2018-09-16 13:50:22 Iter 315300 [Train]: loss=37.59, epe=2.37, lr=0.000025, samples/sec=32.0, sec/step=1.000, eta=3 days, 7:06:10
2018-09-16 13:52:10 Iter 315400 [Train]: loss=37.78, epe=2.39, lr=0.000025, samples/sec=31.5, sec/step=1.017, eta=3 days, 8:23:36
2018-09-16 13:53:59 Iter 315500 [Train]: loss=37.13, epe=2.35, lr=0.000025, samples/sec=31.3, sec/step=1.022, eta=3 days, 8:47:22
2018-09-16 13:55:48 Iter 315600 [Train]: loss=39.32, epe=2.51, lr=0.000025, samples/sec=31.2, sec/step=1.026, eta=3 days, 9:03:26
2018-09-16 13:57:37 Iter 315700 [Train]: loss=37.49, epe=2.38, lr=0.000025, samples/sec=31.1, sec/step=1.028, eta=3 days, 9:11:00
2018-09-16 13:59:26 Iter 315800 [Train]: loss=38.41, epe=2.44, lr=0.000025, samples/sec=31.2, sec/step=1.024, eta=3 days, 8:51:20
2018-09-16 14:01:15 Iter 315900 [Train]: loss=37.57, epe=2.37, lr=0.000025, samples/sec=31.1, sec/step=1.029, eta=3 days, 9:11:02
2018-09-16 14:03:04 Iter 316000 [Train]: loss=36.52, epe=2.29, lr=0.000025, samples/sec=31

2018-09-16 15:25:30 Iter 320500 [Train]: loss=36.32, epe=2.28, lr=0.000025, samples/sec=31.8, sec/step=1.007, eta=3 days, 6:09:08
2018-09-16 15:27:18 Iter 320600 [Train]: loss=37.05, epe=2.33, lr=0.000025, samples/sec=31.4, sec/step=1.018, eta=3 days, 7:02:44
2018-09-16 15:29:07 Iter 320700 [Train]: loss=38.51, epe=2.42, lr=0.000025, samples/sec=31.2, sec/step=1.024, eta=3 days, 7:27:50
2018-09-16 15:30:56 Iter 320800 [Train]: loss=38.03, epe=2.41, lr=0.000025, samples/sec=31.1, sec/step=1.028, eta=3 days, 7:44:06
2018-09-16 15:32:45 Iter 320900 [Train]: loss=36.10, epe=2.26, lr=0.000025, samples/sec=31.0, sec/step=1.031, eta=3 days, 7:57:22
2018-09-16 15:34:35 Iter 321000 [Train]: loss=37.46, epe=2.37, lr=0.000025, samples/sec=31.2, sec/step=1.027, eta=3 days, 7:33:32
2018-09-16 15:34:53 Iter 321000 [Val]: loss=43.23, epe=2.80
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-321000 is not in all_model_checkpoint_paths. Manually adding it.
... mode

2018-09-16 17:02:34 Iter 325700 [Train]: loss=38.26, epe=2.43, lr=0.000025, samples/sec=31.6, sec/step=1.012, eta=3 days, 5:07:29
2018-09-16 17:04:23 Iter 325800 [Train]: loss=36.34, epe=2.29, lr=0.000025, samples/sec=31.3, sec/step=1.022, eta=3 days, 5:49:21
2018-09-16 17:06:12 Iter 325900 [Train]: loss=36.64, epe=2.31, lr=0.000025, samples/sec=31.2, sec/step=1.026, eta=3 days, 6:05:35
2018-09-16 17:08:01 Iter 326000 [Train]: loss=37.63, epe=2.38, lr=0.000025, samples/sec=31.1, sec/step=1.028, eta=3 days, 6:14:26
2018-09-16 17:08:20 Iter 326000 [Val]: loss=41.50, epe=2.69
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-326000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-326000
2018-09-16 17:10:27 Iter 326100 [Train]: loss=36.29, epe=2.28, lr=0.000025, samples/sec=31.3, sec/step=1.022, eta=3 days, 5:47:38
2018-09-16 17:12:16 Iter 326200 [Train]: loss=37.15, epe=

## Training log

TBD: Here are the training curves for the run above:

![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/loss.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/epe.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/lr.png)

TBD: Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val1.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val2.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val3.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val4.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val5.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val6.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val7.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val8.png)